In [6]:
#setting up GPUs
from typing import Callable, List, Tuple 

import os
import torch
import catalyst

from catalyst.dl import utils
SEED = 42
utils.set_global_seed(SEED)
utils.prepare_cudnn(deterministic=True)
print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

torch: 1.4.0, catalyst: 20.05


In [29]:
import data_processing
path_to_csv="/data/mialab/users/AA/Zafar/AA_adni_smri_tbl.csv"
path_to_nifti="/data/mialab/users/AA/Zafar/adni_smri/"

d_process=data_processing.data_preprocssing(path_to_csv, path_to_nifti)
train_data,test_data=d_process.build()

In [30]:
print('Dataset size:', len(train_data), 'subjects')
print('Dataset size:', len(test_data), 'subjects')

Dataset size: 662 subjects
Dataset size: 166 subjects


In [31]:
type(train_data[0]['features'])

torchio.data.image.Image

In [32]:
import collections
training_loader = DataLoader(train_data, batch_size=4, num_workers=4)
validation_loader = DataLoader(test_data, batch_size=4, num_workers=4)
loaders = collections.OrderedDict()
loaders["train"] = training_loader
loaders["valid"] = validation_loader

In [33]:
for a,b in enumerate(training_loader):
    print(b)
    if a==1:
        break

{'features': {'data': tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          ...,

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
     

In [35]:
for i,j in enumerate(loaders['train']):
  
  print(type((j['features'])))
  #print(i,j['image'])
  #print(i,j['label'].size())
  #print(j['targets'])
  if i==1:
    break

<class 'dict'>
<class 'dict'>


In [36]:
# this variable will be used in `runner.train` and by default we disable FP16 mode
is_fp16_used = False
is_alchemy_used = False

In [14]:
# # for augmentations
# !pip install albumentations

# # for pretrained models for PyTorch
# !pip install pretrainedmodels


# ################
# # Catalyst itself
# !pip install -U catalyst
# # For specific version of catalyst, uncomment:
# # ! pip install git+http://github.com/catalyst-team/catalyst.git@{master/commit_hash}
# ################

# # for TTA
# # !pip install ttach

# # for tensorboard
# !pip install tensorflow
# %load_ext tensorboard

# # for alchemy experiment logging integration, uncomment this 2 lines below
# # !pip install -U alchemy
# # is_alchemy_used = True

# # if Your machine support Apex FP16, uncomment this 3 lines below
# # !git clone https://github.com/NVIDIA/apex
# # !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
# # is_fp16_used = True

In [15]:
#Integrate visualization library to colab
import IPython

def configure_plotly_browser_state():
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))


IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

In [37]:
import model
model=model.ResNet_l4(model.BasicBlock,[2, 2, 2, 2],4)
print(model)

ResNet_l4(
  (conv1): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [38]:
num_classes = 4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[9], gamma=0.3
)


from catalyst.dl import SupervisedRunner

num_epochs = 10
logdir = "./logs/classification"

device = utils.get_device()
print(f"device: {device}")

if is_fp16_used:
    fp16_params = dict(opt_level="O1") # params for FP16
else:
    fp16_params = None

print(f"FP16 params: {fp16_params}")

runner = SupervisedRunner(device=device)



device: cpu
FP16 params: None


In [42]:
# as we are working on classification task
from catalyst.dl.callbacks import AccuracyCallback, AUCCallback, F1ScoreCallback

callbacks = [
    AccuracyCallback(num_classes=num_classes),
    AUCCallback(
        num_classes=num_classes,
        input_key="targets_one_hot"
    ),
    F1ScoreCallback(
        input_key="targets_one_hot",
        activation="Softmax"
    )
]

if is_alchemy_used:
    from catalyst.dl import AlchemyLogger
    callbacks.append(AlchemyLogger(**monitoring_params))

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    # our dataloaders
    loaders=loaders,
    # We can specify the callbacks list for the experiment;
    # For this task, we will check accuracy, AUC and F1 metrics
    callbacks=callbacks,
    # path to save logs
    logdir=logdir,
    num_epochs=num_epochs,
    # save our best checkpoint by AUC metric
    main_metric="auc/_mean",
    # AUC needs to be maximized.
    minimize_metric=False,
    # for FP16. It uses the variable from the very first cell
    fp16=fp16_params,
    # prints train logs
    verbose=True,
)



1/10 * Epoch (train):   0% 0/166 [00:00<?, ?it/s]

TypeError: conv3d(): argument 'input' (position 1) must be Tensor, not dict